<a href="https://colab.research.google.com/github/yungshan629/NTU-Discrete-Mathematics/blob/main/%E9%9B%A2%E6%95%A3%E6%95%B8%E5%AD%B8%E4%BD%9C%E6%A5%AD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 定義海戰的基本資料

為了讓作業能夠順利進行，
我要將程式碼分成兩大部分，
第一部分是基本的設定，
包括網格大小，船艦類型，飛彈，船艦配置，判斷勝利條件，主遊戲邏輯。
第二部分是攻擊與防守，
這部分的重點是攻擊策略、防守策略
第三部分是模擬及計算，
包括存活率、擊沈率、"賽局策略？"這些東西。
第四部分有餘力再作，看看能不能視覺化。

第一部分和第三部分要先完成，不然會一天過去一直原地踏步。
第二部分再慢慢思考要如何改進。

In [ ]:
import random
import numpy as np

# 定義網格大小
GRID_SIZE = 20

# 定義船艦的類型
class Ship:
    def __init__(self, name: str, size: str, speed: int, health: int, sonic_missiles: int, hypersonic_missiles: int):
        self.name = name
        self.size = size
        self.speed = speed
        self.health = health
        self.sonic_missiles = sonic_missiles
        self.hypersonic_missiles = hypersonic_missiles
        self.position = (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1))

    def move(self, direction):
        x, y = self.position
        if direction == 'up':
            self.position = (x, max(0, y - self.speed))
        elif direction == 'down':
            self.position = (x, min(GRID_SIZE-1, y + self.speed))
        elif direction == 'left':
            self.position = (max(0, x - self.speed), y)
        elif direction == 'right':
            self.position = (min(GRID_SIZE-1, x + self.speed), y)

    def take_damage(self, damage):
        self.health -= damage

    def is_sunk(self):
        return self.health <= 0

# 定義飛彈類型
class Missile:
    def __init__(self, missile_type):
        self.missile_type = missile_type
        if missile_type == 'sonic':
            self.range = 1  # 十字範圍
        elif missile_type == 'hypersonic':
            self.range = 1  # 九宮格範圍

    def attack(self, attacker_pos, target_pos):
        # 計算距離
        x1, y1 = attacker_pos
        x2, y2 = target_pos
        dx = abs(x1 - x2)
        dy = abs(y1 - y2)

        if self.missile_type == 'sonic':
            return dx <= self.range or dy <= self.range  # 十字範圍
        elif self.missile_type == 'hypersonic':
            return dx <= self.range and dy <= self.range  # 九宮格範圍

# 生成船艦的配置
def create_ships():
    ships = [
        Ship("Large", "large", 6, 3, 5, 4),  # 大型船艦
        Ship("Medium", "medium", 4, 2, 4, 2),  # 中型船艦
        Ship("Small", "small", 3, 1, 0, 3),  # 小型船艦
    ]
    return ships

# 生成雙方的船艦
def setup_game():
    team_a_ships = create_ships()  # 甲方
    team_b_ships = create_ships()  # 乙方
    return team_a_ships, team_b_ships

# 印出船艦位置
def print_ship_positions(ships):
    for ship in ships:
        print(f'{ship.name} at position {ship.position}')

# 選擇攻擊目標函數
def choose_target(attacker, enemy_ships):
    if not enemy_ships:
        return None
    # 選擇血量最低的目標
    return min(enemy_ships, key=lambda ship: ship.health)

# 判斷勝利條件
def check_winner(team_a_ships, team_b_ships):
    if not team_a_ships:
        print("Team B wins!")
        return True
    elif not team_b_ships:
        print("Team A wins!")
        return True
    return False

# 主遊戲邏輯
def game_turn(team_a_ships, team_b_ships):
    # 簡單的攻擊邏輯示例，隊伍A攻擊隊伍B
    for attacker in team_a_ships:
        if team_b_ships:
            target = choose_target(attacker, team_b_ships)
            if target:
                missile_type = 'sonic'  # 假設選擇音速飛彈
                missile = Missile(missile_type)
                if missile.attack(attacker.position, target.position):
                    target.take_damage(1)
                    if target.is_sunk():
                        print(f"{target.name} is sunk!")
                        team_b_ships.remove(target)

# 修改的防守策略函數
def defensive_move(ship, enemy_positions):
    # 簡單策略：隨機移動以避開敵人
    directions = ['up', 'down', 'left', 'right']
    ship.move(random.choice(directions))





# 定義決策矩陣


In [ ]:


# 定义网格大小和回合数

MAX_ROUNDS = 50
SIMULATION_COUNT = 100  # 多次模拟的次数


# 定义策略空间，使用矩阵来表示
# 假设策略向量包含：船舰类型选择 (0-2)，目标选择策略 (0: random, 1: lowest health)，飞弹选择策略 (0: sonic, 1: hypersonic)
strategy_space = [
    [0, 1, 0],  # 大型船舰，优先攻击血量最低的目标，使用音速飞弹
    [1, 0, 1],  # 中型船舰，随机攻击目标，使用超音速飞弹
    [2, 1, 0],  # 小型船舰，优先攻击血量最低的目标，使用音速飞弹
    # 可以扩展更多组合
]

# 定义评估函数，计算击毁率和存活率
def evaluate_strategy(strategy):
    total_destroyed = 0
    total_survived = 0

    for _ in range(SIMULATION_COUNT):
        team_a_ships, team_b_ships = setup_game()
        round_count = 0

        while round_count < MAX_ROUNDS:
            for attacker in team_a_ships:
                target = choose_target(attacker, team_b_ships) if strategy[1] == 1 else random.choice(team_b_ships)
                missile_type = 'sonic' if strategy[2] == 0 else 'hypersonic'

                if missile_type and target:
                    missile = Missile(missile_type)
                    if missile.attack(attacker.position, target.position):
                        target.take_damage(1)
                        if target.is_sunk():
                            team_b_ships.remove(target)

            # 防守逻辑可根据策略调整
            for ship in team_a_ships:
                defensive_move(ship, [s.position for s in team_b_ships])

            # 判断游戏是否结束
            if check_winner(team_a_ships, team_b_ships):
                break

            round_count += 1

        # 记录战局结果
        total_destroyed += len([ship for ship in team_b_ships if ship.is_sunk()])
        total_survived += len(team_a_ships)

    # 计算平均击毁率和存活率
    avg_destroyed = total_destroyed / SIMULATION_COUNT
    avg_survived = total_survived / SIMULATION_COUNT
    return avg_destroyed, avg_survived

# 评估所有策略组合
strategy_results = []
for strategy in strategy_space:
    destroyed, survived = evaluate_strategy(strategy)
    strategy_results.append((strategy, destroyed, survived))

# 找到击毁率和存活率最高的策略组合
best_destroy_strategy = max(strategy_results, key=lambda x: x[1])
best_survive_strategy = max(strategy_results, key=lambda x: x[2])

# 输出结果
print(f'最佳击毁策略：{best_destroy_strategy[0]}，平均击毁率：{best_destroy_strategy[1]}')
print(f'最佳存活策略：{best_survive_strategy[0]}，平均存活率：{best_survive_strategy[2]}')






Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!
Team A wins!

IndexError: list index out of range

In [ ]:
# 主遊戲邏輯
def main_game():
    team_a_ships, team_b_ships = setup_game()
    round_count = 0

    while round_count < MAX_ROUNDS:  # 設定最多回合數
        print(f'--- Round {round_count + 1} ---')
        print('Team A ships:')
        print_ship_positions(team_a_ships)
        print('Team B ships:')
        print_ship_positions(team_b_ships)

        # 甲方回合攻擊乙方
        game_turn(team_a_ships, team_b_ships)

        # 乙方回合攻擊甲方
        game_turn(team_b_ships, team_a_ships)

        # 檢查勝利條件
        if check_winner(team_a_ships, team_b_ships):
            break

        round_count += 1

    else:
        print("Game ended in a draw.")



In [ ]:
import numpy as np

# 定義網格大小
GRID_SIZE = 10  # 例如，20x20 的網格

# 定義船艦類別
class Ship:
    def __init__(self, ship_type, position, health, speed, sonic_missiles, hypersonic_missiles):
        self.ship_type = ship_type
        self.position = position
        self.health = health
        self.speed = speed
        self.sonic_missiles = sonic_missiles
        self.hypersonic_missiles = hypersonic_missiles
        self.is_sunk = False

    def move(self, new_position):
        if not self.is_sunk:
            self.position = new_position

    def take_hit(self):
        if not self.is_sunk:
            self.health -= 1
            if self.health <= 0:
                self.is_sunk = True

# 初始化船艦
def initialize_ships():
    ships = {
        'A': [
            Ship('S_large', (0, 0), 3, 4, 5, 4),
            Ship('S_large', (1, 1), 3, 4, 5, 4),
            Ship('S_large', (2, 2), 3, 4, 5, 4),
            Ship('S_large', (3, 3), 3, 4, 5, 4),
            Ship('S_medium', (4, 4), 2, 3, 4, 2),
            Ship('S_medium', (5, 5), 2, 3, 4, 2),
            Ship('S_small', (6, 6), 1, 2, 0, 3),
            Ship('S_small', (7, 7), 1, 2, 0, 3)
        ],
        'B': [
            Ship('S_large', (10, 10), 3, 4, 5, 4),
            Ship('S_large', (11, 11), 3, 4, 5, 4),
            Ship('S_large', (12, 12), 3, 4, 5, 4),
            Ship('S_medium', (13, 13), 2, 3, 4, 2),
            Ship('S_medium', (14, 14), 2, 3, 4, 2),
            Ship('S_medium', (15, 15), 2, 3, 4, 2),
            Ship('S_small', (16, 16), 1, 2, 0, 3),
            Ship('S_small', (17, 17), 1, 2, 0, 3),
            Ship('S_small', (18, 18), 1, 2, 0, 3)
        ]
    }
    return ships

# 限制船艦移動在網格內的方法
def move_within_grid(ship, new_position):
    x, y = new_position
    x = max(0, min(x, GRID_SIZE - 1))  # 確保 x 在 0 到 GRID_SIZE-1 之間
    y = max(0, min(y, GRID_SIZE - 1))  # 確保 y 在 0 到 GRID_SIZE-1 之間
    ship.move((x, y))


# 計算兩艘船之間的距離
def calculate_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

# 檢查是否在攻擊範圍內
def is_in_attack_range(attacker, target, missile_type):
    x1, y1 = attacker.position
    x2, y2 = target.position

    if missile_type == 'sonic':
        # 十字範圍：同一列或同一行且距離 <= 1
        return (x1 == x2 and abs(y1 - y2) <= 1) or (y1 == y2 and abs(x1 - x2) <= 1)
    elif missile_type == 'hypersonic':
        # 九宮格範圍：包括攻擊者位置周圍的3x3範圍
        return abs(x1 - x2) <= 1 and abs(y1 - y2) <= 1
    return False



# 定義攻擊策略
def attack_strategy(attacker_ships, target_ships):
    for attacker in attacker_ships:
        if attacker.is_sunk:
            continue
        for missile_type in ['sonic', 'hypersonic']:
            if missile_type == 'sonic' and attacker.sonic_missiles > 0:
                attack_type = missile_type
            elif missile_type == 'hypersonic' and attacker.hypersonic_missiles > 0:
                attack_type = missile_type
            else:
                continue  # 如果沒有飛彈可用，跳過

            for target in target_ships:
                if not target.is_sunk:
                    print(f"Checking attack range from {attacker.ship_type} at {attacker.position} to {target.ship_type} at {target.position}")
                    if is_in_attack_range(attacker, target, attack_type):
                        print(f"{attacker.ship_type} at {attacker.position} uses {attack_type} missile to attack {target.ship_type} at {target.position}")
                        target.take_hit()
                        if target.is_sunk:
                            print(f"{target.ship_type} has been sunk!")
                        # 扣除使用的飛彈
                        if attack_type == 'sonic':
                            attacker.sonic_missiles -= 1
                        else:
                            attacker.hypersonic_missiles -= 1
                        break  # 攻擊一次後退出內層循環



# 修改防守策略方法
def defense_strategy(ships):
    for ship in ships:
        if not ship.is_sunk:
            # 簡單策略：向隨機方向移動其速度的格數（這裡用速度增加位置）
            new_position = (ship.position[0] + ship.speed, ship.position[1] + ship.speed)
            move_within_grid(ship, new_position)
            print(f"{ship.ship_type} moves to {ship.position}")

# 建立賽局矩陣：代表雙方不同策略的結果
def create_game_matrix():
    matrix = np.zeros((3, 3))  # 簡化為3x3矩陣
    # 填入矩陣值表示策略組合的效果
    matrix[0, 0] = 1  # 攻擊策略1 vs 防守策略1
    matrix[0, 1] = -1
    matrix[0, 2] = 0
    matrix[1, 0] = 0
    matrix[1, 1] = 1
    matrix[1, 2] = -1
    matrix[2, 0] = -1
    matrix[2, 1] = 0
    matrix[2, 2] = 1
    return matrix

# 模擬遊戲流程
def simulate_game(rounds=30):
    ships = initialize_ships()
    for round_number in range(1, rounds + 1):
        print(f"\nRound {round_number} - Attack Phase")
        attack_strategy(ships['A'], ships['B'])
        print(f"\nRound {round_number} - Defense Phase")
        defense_strategy(ships['B'])
        print(f"\nEnd of Round {round_number}")

    print("\nGame Matrix:")
    print(create_game_matrix())

# 執行模擬
simulate_game()



Round 1 - Attack Phase
Checking attack range from S_large at (0, 0) to S_large at (10, 10)
Checking attack range from S_large at (0, 0) to S_large at (11, 11)
Checking attack range from S_large at (0, 0) to S_large at (12, 12)
Checking attack range from S_large at (0, 0) to S_medium at (13, 13)
Checking attack range from S_large at (0, 0) to S_medium at (14, 14)
Checking attack range from S_large at (0, 0) to S_medium at (15, 15)
Checking attack range from S_large at (0, 0) to S_small at (16, 16)
Checking attack range from S_large at (0, 0) to S_small at (17, 17)
Checking attack range from S_large at (0, 0) to S_small at (18, 18)
Checking attack range from S_large at (0, 0) to S_large at (10, 10)
Checking attack range from S_large at (0, 0) to S_large at (11, 11)
Checking attack range from S_large at (0, 0) to S_large at (12, 12)
Checking attack range from S_large at (0, 0) to S_medium at (13, 13)
Checking attack range from S_large at (0, 0) to S_medium at (14, 14)
Checking attack ran

In [23]:
import numpy as np
from enum import Enum
from dataclasses import dataclass
from typing import List, Tuple, Dict
import random

# 定義船艦類型
class ShipType(Enum):
    LARGE = "large"
    MEDIUM = "medium"
    SMALL = "small"

# 定義飛彈類型
class MissileType(Enum):
    SONIC = "sonic"
    HYPERSONIC = "hypersonic"

@dataclass
class Ship:
    type: ShipType
    position: Tuple[int, int]
    health: int
    sonic_missiles: int
    hypersonic_missiles: int
    speed: int

    @classmethod
    def create(cls, ship_type: ShipType, position: Tuple[int, int]):
        if ship_type == ShipType.LARGE:
            return cls(ship_type, position, 3, 5, 4, 4)
        elif ship_type == ShipType.MEDIUM:
            return cls(ship_type, position, 2, 4, 2, 3)
        else:  # SMALL
            return cls(ship_type, position, 1, 0, 3, 2)

class NavalBattleGame:
    def __init__(self):
        self.grid_size = 10
        self.grid = np.zeros((self.grid_size, self.grid_size))

        # 初始化甲方艦隊
        self.team_A = {
            ShipType.LARGE: [Ship.create(ShipType.LARGE, (0, i)) for i in range(4)],
            ShipType.MEDIUM: [Ship.create(ShipType.MEDIUM, (1, i)) for i in range(2)],
            ShipType.SMALL: [Ship.create(ShipType.SMALL, (2, i)) for i in range(2)]
        }

        # 初始化乙方艦隊
        self.team_B = {
            ShipType.LARGE: [Ship.create(ShipType.LARGE, (9, i)) for i in range(3)],
            ShipType.MEDIUM: [Ship.create(ShipType.MEDIUM, (8, i)) for i in range(3)],
            ShipType.SMALL: [Ship.create(ShipType.SMALL, (7, i)) for i in range(3)]
        }

        # 初始化策略矩陣
        self.strategy_matrix = np.zeros((3, 3))  # 3種船型 x 3種攻擊策略

    def calculate_attack_matrix(self) -> np.ndarray:
        """計算攻擊效率矩陣"""
        attack_matrix = np.array([
            [0.7, 0.8, 0.9],  # 大型船艦攻擊效率 [大,中,小]
            [0.6, 0.7, 0.8],  # 中型船艦攻擊效率
            [0.5, 0.6, 0.7]   # 小型船艦攻擊效率
        ])
        return attack_matrix

    def calculate_defense_matrix(self) -> np.ndarray:
        """計算防禦效率矩陣"""
        defense_matrix = np.array([
            [0.6, 0.5, 0.4],  # 大型船艦防禦效率
            [0.5, 0.4, 0.3],  # 中型船艦防禦效率
            [0.4, 0.3, 0.2]   # 小型船艦防禦效率
        ])
        return defense_matrix

    def calculate_game_matrix(self) -> np.ndarray:
        """計算賽局矩陣"""
        attack_matrix = self.calculate_attack_matrix()
        defense_matrix = self.calculate_defense_matrix()

        # 綜合攻防效率
        game_matrix = attack_matrix - defense_matrix
        return game_matrix

    def get_missile_hit_area(self, position: Tuple[int, int], missile_type: MissileType) -> List[Tuple[int, int]]:
        """計算飛彈攻擊範圍"""
        x, y = position
        hit_area = []

        if missile_type == MissileType.SONIC:
            # 音速飛彈：十字形
            for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
                new_x, new_y = x + dx, y + dy
                if 0 <= new_x < self.grid_size and 0 <= new_y < self.grid_size:
                    hit_area.append((new_x, new_y))

        else:  # HYPERSONIC
            # 超音速飛彈：九宮格
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    new_x, new_y = x + dx, y + dy
                    if 0 <= new_x < self.grid_size and 0 <= new_y < self.grid_size:
                        hit_area.append((new_x, new_y))

        return hit_area

    def calculate_survival_rate(self, team: Dict) -> Dict[ShipType, float]:
        """計算船艦存活率"""
        survival_rates = {}
        for ship_type in ShipType:
            ships = team[ship_type]
            total_ships = len(ships)
            alive_ships = len([ship for ship in ships if ship.health > 0])
            survival_rates[ship_type] = alive_ships / total_ships if total_ships > 0 else 0
        return survival_rates

    def calculate_destruction_rate(self, attacker: Dict, defender: Dict) -> Dict[ShipType, float]:
        """計算擊毀率"""
        destruction_rates = {}
        attack_matrix = self.calculate_attack_matrix()

        for ship_type in ShipType:
            # 獲取攻擊方該類型船艦的攻擊效率
            attack_efficiency = attack_matrix[list(ShipType).index(ship_type)]

            # 計算總攻擊力
            total_attack_power = sum(ship.sonic_missiles + ship.hypersonic_missiles
                                   for ship in attacker[ship_type])

            # 計算預期擊毀數
            expected_destructions = total_attack_power * attack_efficiency.mean()

            # 計算擊毀率
            total_defender_ships = len(defender[ship_type])
            destruction_rates[ship_type] = min(1.0, expected_destructions / total_defender_ships) if total_defender_ships > 0 else 0

        return destruction_rates

    def simulate_battle(self, rounds: int = 3) -> Dict:
        """模擬戰鬥過程"""
        results = {
            'team_A_survival': [],
            'team_B_survival': [],
            'team_A_destruction': [],
            'team_B_destruction': []
        }

        for round_num in range(rounds):
            # 計算存活率
            team_A_survival = self.calculate_survival_rate(self.team_A)
            team_B_survival = self.calculate_survival_rate(self.team_B)

            # 計算擊毀率
            team_A_destruction = self.calculate_destruction_rate(self.team_A, self.team_B)
            team_B_destruction = self.calculate_destruction_rate(self.team_B, self.team_A)

            # 記錄結果
            results['team_A_survival'].append(team_A_survival)
            results['team_B_survival'].append(team_B_survival)
            results['team_A_destruction'].append(team_A_destruction)
            results['team_B_destruction'].append(team_B_destruction)

            # 模擬攻擊效果
            self._simulate_attack_effects()

        return results

    def _simulate_attack_effects(self):
        """模擬攻擊效果"""
        game_matrix = self.calculate_game_matrix()

        # 隨機選擇攻擊目標並計算傷害
        for team in [self.team_A, self.team_B]:
            for ship_type in ShipType:
                for ship in team[ship_type]:
                    if ship.health <= 0:
                        continue

                    # 模擬傷害
                    damage_prob = game_matrix[list(ShipType).index(ship_type)].mean()
                    if random.random() < damage_prob:
                        ship.health -= 1

# 測試程式
def main():
    game = NavalBattleGame()

    # 模擬戰鬥
    results = game.simulate_battle()

    # 輸出結果
    print("\n=== 戰鬥模擬結果 ===")

    for round_num in range(3):
        print(f"\n第 {round_num + 1} 回合:")
        print("\n甲方存活率:")
        for ship_type, rate in results['team_A_survival'][round_num].items():
            print(f"{ship_type.value}: {rate*100:.1f}%")

        print("\n乙方存活率:")
        for ship_type, rate in results['team_B_survival'][round_num].items():
            print(f"{ship_type.value}: {rate*100:.1f}%")

        print("\n甲方擊毀率:")
        for ship_type, rate in results['team_A_destruction'][round_num].items():
            print(f"{ship_type.value}: {rate*100:.1f}%")

        print("\n乙方擊毀率:")
        for ship_type, rate in results['team_B_destruction'][round_num].items():
            print(f"{ship_type.value}: {rate*100:.1f}%")

if __name__ == "__main__":
    main()


=== 戰鬥模擬結果 ===

第 1 回合:

甲方存活率:
large: 100.0%
medium: 100.0%
small: 100.0%

乙方存活率:
large: 100.0%
medium: 100.0%
small: 100.0%

甲方擊毀率:
large: 100.0%
medium: 100.0%
small: 100.0%

乙方擊毀率:
large: 100.0%
medium: 100.0%
small: 100.0%

第 2 回合:

甲方存活率:
large: 100.0%
medium: 100.0%
small: 50.0%

乙方存活率:
large: 100.0%
medium: 100.0%
small: 100.0%

甲方擊毀率:
large: 100.0%
medium: 100.0%
small: 100.0%

乙方擊毀率:
large: 100.0%
medium: 100.0%
small: 100.0%

第 3 回合:

甲方存活率:
large: 100.0%
medium: 100.0%
small: 50.0%

乙方存活率:
large: 100.0%
medium: 66.7%
small: 66.7%

甲方擊毀率:
large: 100.0%
medium: 100.0%
small: 100.0%

乙方擊毀率:
large: 100.0%
medium: 100.0%
small: 100.0%
